In [0]:
storageAccountName = 'equitymarketstorage'
storageAccountAccessKey = '32Q+pSDygN/vBS/+2OPfjXJrW9+T4gogsOJY3mEob/GON/Hls27FX1ZaxbBxEjsmA+/+XJn1AXkFj2dUSrOeWA=='
blobContainerName = 'firstcontainer'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")
    
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/spark_data")) 

path,name,size
dbfs:/mnt/FileStore/MountFolder/spark_data/answers/,answers/,0
dbfs:/mnt/FileStore/MountFolder/spark_data/questions/,questions/,0


In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, month
import os

answersDF = spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/spark_data/answers/*")

questionsDF = spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/spark_data/questions/*")

'''
Answers aggregation

Here we : get number of answers per question per month
'''

answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month').agg(count('*').alias('cnt'))

resultDF = questionsDF.join(answers_month, 'question_id').select('question_id', 'creation_date', 'title', 'month', 'cnt')

resultDF.orderBy('question_id', 'month').show()

'''
Task:

see the query plan of the previous result and rewrite the query to optimize it
'''

+-----------+--------------------+--------------------+-----+---+
question_id| creation_date| title|month|cnt|
+-----------+--------------------+--------------------+-----+---+
 155989|2015-01-01 01:59:...|Frost bubble form...| 1| 1|
 155989|2015-01-01 01:59:...|Frost bubble form...| 2| 1|
 155990|2015-01-01 02:51:...|The abstract spac...| 1| 2|
 155992|2015-01-01 03:44:...|centrifugal force...| 1| 1|
 155993|2015-01-01 03:56:...|How can I estimat...| 1| 1|
 155995|2015-01-01 05:16:...|Why should a solu...| 1| 3|
 155996|2015-01-01 06:06:...|Why do we assume ...| 1| 2|
 155996|2015-01-01 06:06:...|Why do we assume ...| 2| 1|
 155996|2015-01-01 06:06:...|Why do we assume ...| 11| 1|
 155997|2015-01-01 06:26:...|Why do square sha...| 1| 3|
 155999|2015-01-01 07:01:...|Diagonalizability...| 1| 1|
 156008|2015-01-01 08:48:...|Capturing a light...| 1| 2|
 156008|2015-01-01 08:48:...|Capturing a light...| 11| 1|
 156016|2015-01-01 10:31:...|The interference ...| 1| 1|
 156020|2015-01-01 11:19:...|What is going on ...| 1| 1|
 156021|2015-01-01 11:21:...|How to calculate ...| 2| 1|
 156022|2015-01-01 11:55:...|Advice on Major S...| 1| 1|
 156025|2015-01-01 12:32:...|Deriving the Cano...| 1| 1|
 156026|2015-01-01 12:49:...|Does Bell's inequ...| 1| 3|
 156027|2015-01-01 12:49:...|Deriving X atom f...| 1| 1|
+-----------+--------------------+--------------------+-----+---+
only showing top 20 rows

--- 2.082385301589966 seconds ---
Out[9]: '\nTask:\n\nsee the query plan of the previous result and rewrite the query to optimize it\n'

In [0]:
resultDF.explain("FORMATTED")

== Physical Plan ==
AdaptiveSparkPlan (12)
+- Project (11)
 +- BroadcastHashJoin Inner BuildRight (10)
 :- Filter (2)
 : +- Scan parquet (1)
 +- BroadcastExchange (9)
 +- HashAggregate (8)
 +- Exchange (7)
 +- HashAggregate (6)
 +- Project (5)
 +- Filter (4)
 +- Scan parquet (3)


(1) Scan parquet 
Output [3]: [question_id#312L, creation_date#314, title#315]
Batched: true
Location: InMemoryFileIndex [dbfs:/mnt/FileStore/MountFolder/spark_data/questions/_SUCCESS, ... 4 entries]
PushedFilters: [IsNotNull(question_id)]
ReadSchema: struct<question_id:bigint,creation_date:timestamp,title:string>

(2) Filter
Input [3]: [question_id#312L, creation_date#314, title#315]
Condition : isnotnull(question_id#312L)

(3) Scan parquet 
Output [2]: [question_id#300L, creation_date#302]
Batched: true
Location: InMemoryFileIndex [dbfs:/mnt/FileStore/MountFolder/spark_data/answers/_SUCCESS, ... 4 entries]
PushedFilters: [IsNotNull(question_id)]
ReadSchema: struct<question_id:bigint,creation_date:timestamp>

(4) Filter
Input [2]: [question_id#300L, creation_date#302]
Condition : isnotnull(question_id#300L)

(5) Project
Output [2]: [question_id#300L, month(cast(creation_date#302 as date)) AS month#328]
Input [2]: [question_id#300L, creation_date#302]

(6) HashAggregate
Input [2]: [question_id#300L, month#328]
Keys [2]: [question_id#300L, month#328]
Functions [1]: [partial_count(1) AS count#379L]
Aggregate Attributes [1]: [count#378L]
Results [3]: [question_id#300L, month#328, count#379L]

(7) Exchange
Input [3]: [question_id#300L, month#328, count#379L]
Arguments: hashpartitioning(question_id#300L, month#328, 200), ENSURE_REQUIREMENTS, [id=#1025]

(8) HashAggregate
Input [3]: [question_id#300L, month#328, count#379L]
Keys [2]: [question_id#300L, month#328]
Functions [1]: [finalmerge_count(merge count#379L) AS count(1)#343L]
Aggregate Attributes [1]: [count(1)#343L]
Results [3]: [question_id#300L, month#328, count(1)#343L AS cnt#344L]

(9) BroadcastExchange
Input [3]: [question_id#300L, month#328, cnt#344L]
Arguments: HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [id=#1028]

(10) BroadcastHashJoin
Left keys [1]: [question_id#312L]
Right keys [1]: [question_id#300L]
Join condition: None

(11) Project
Output [5]: [question_id#312L, creation_date#314, title#315, month#328, cnt#344L]
Input [6]: [question_id#312L, creation_date#314, title#315, question_id#300L, month#328, cnt#344L]

(12) AdaptiveSparkPlan
Output [5]: [question_id#312L, creation_date#314, title#315, month#328, cnt#344L]
Arguments: isFinalPlan=false

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, month
import os

spark.conf.set("spark.sql.adaptive.enabled","true")
answersDF = spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/spark_data/answers/*")

questionsDF = spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/spark_data/questions/*")

answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month').agg(count('*').alias('cnt'))

resultDF = questionsDF.join(answers_month, 'question_id').select('question_id', 'creation_date', 'title', 'month', 'cnt')

resultDF.orderBy('question_id', 'month').show()


+-----------+--------------------+--------------------+-----+---+
question_id| creation_date| title|month|cnt|
+-----------+--------------------+--------------------+-----+---+
 155989|2015-01-01 01:59:...|Frost bubble form...| 1| 1|
 155989|2015-01-01 01:59:...|Frost bubble form...| 2| 1|
 155990|2015-01-01 02:51:...|The abstract spac...| 1| 2|
 155992|2015-01-01 03:44:...|centrifugal force...| 1| 1|
 155993|2015-01-01 03:56:...|How can I estimat...| 1| 1|
 155995|2015-01-01 05:16:...|Why should a solu...| 1| 3|
 155996|2015-01-01 06:06:...|Why do we assume ...| 1| 2|
 155996|2015-01-01 06:06:...|Why do we assume ...| 2| 1|
 155996|2015-01-01 06:06:...|Why do we assume ...| 11| 1|
 155997|2015-01-01 06:26:...|Why do square sha...| 1| 3|
 155999|2015-01-01 07:01:...|Diagonalizability...| 1| 1|
 156008|2015-01-01 08:48:...|Capturing a light...| 1| 2|
 156008|2015-01-01 08:48:...|Capturing a light...| 11| 1|
 156016|2015-01-01 10:31:...|The interference ...| 1| 1|
 156020|2015-01-01 11:19:...|What is going on ...| 1| 1|
 156021|2015-01-01 11:21:...|How to calculate ...| 2| 1|
 156022|2015-01-01 11:55:...|Advice on Major S...| 1| 1|
 156025|2015-01-01 12:32:...|Deriving the Cano...| 1| 1|
 156026|2015-01-01 12:49:...|Does Bell's inequ...| 1| 3|
 156027|2015-01-01 12:49:...|Deriving X atom f...| 1| 1|
+-----------+--------------------+--------------------+-----+---+
only showing top 20 rows

In [0]:
resultDF.explain("FORMATTED")

== Physical Plan ==
AdaptiveSparkPlan (12)
+- Project (11)
 +- BroadcastHashJoin Inner BuildRight (10)
 :- Filter (2)
 : +- Scan parquet (1)
 +- BroadcastExchange (9)
 +- HashAggregate (8)
 +- Exchange (7)
 +- HashAggregate (6)
 +- Project (5)
 +- Filter (4)
 +- Scan parquet (3)


(1) Scan parquet 
Output [3]: [question_id#584L, creation_date#586, title#587]
Batched: true
Location: InMemoryFileIndex [dbfs:/mnt/FileStore/MountFolder/spark_data/questions/_SUCCESS, ... 4 entries]
PushedFilters: [IsNotNull(question_id)]
ReadSchema: struct<question_id:bigint,creation_date:timestamp,title:string>

(2) Filter
Input [3]: [question_id#584L, creation_date#586, title#587]
Condition : isnotnull(question_id#584L)

(3) Scan parquet 
Output [2]: [question_id#572L, creation_date#574]
Batched: true
Location: InMemoryFileIndex [dbfs:/mnt/FileStore/MountFolder/spark_data/answers/_SUCCESS, ... 4 entries]
PushedFilters: [IsNotNull(question_id)]
ReadSchema: struct<question_id:bigint,creation_date:timestamp>

(4) Filter
Input [2]: [question_id#572L, creation_date#574]
Condition : isnotnull(question_id#572L)

(5) Project
Output [2]: [question_id#572L, month(cast(creation_date#574 as date)) AS month#600]
Input [2]: [question_id#572L, creation_date#574]

(6) HashAggregate
Input [2]: [question_id#572L, month#600]
Keys [2]: [question_id#572L, month#600]
Functions [1]: [partial_count(1) AS count#651L]
Aggregate Attributes [1]: [count#650L]
Results [3]: [question_id#572L, month#600, count#651L]

(7) Exchange
Input [3]: [question_id#572L, month#600, count#651L]
Arguments: hashpartitioning(question_id#572L, month#600, 200), ENSURE_REQUIREMENTS, [id=#1766]

(8) HashAggregate
Input [3]: [question_id#572L, month#600, count#651L]
Keys [2]: [question_id#572L, month#600]
Functions [1]: [finalmerge_count(merge count#651L) AS count(1)#615L]
Aggregate Attributes [1]: [count(1)#615L]
Results [3]: [question_id#572L, month#600, count(1)#615L AS cnt#616L]

(9) BroadcastExchange
Input [3]: [question_id#572L, month#600, cnt#616L]
Arguments: HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [id=#1769]

(10) BroadcastHashJoin
Left keys [1]: [question_id#584L]
Right keys [1]: [question_id#572L]
Join condition: None

(11) Project
Output [5]: [question_id#584L, creation_date#586, title#587, month#600, cnt#616L]
Input [6]: [question_id#584L, creation_date#586, title#587, question_id#572L, month#600, cnt#616L]

(12) AdaptiveSparkPlan
Output [5]: [question_id#584L, creation_date#586, title#587, month#600, cnt#616L]
Arguments: isFinalPlan=false